In [ ]:
import json
from sqlalchemy import create_engine, MetaData, Table, select, and_, func

In [ ]:
sql_credentials = json.load(open('/root/thomas/sqlcredentials.json'))
sql_engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                                         sql_credentials["host"], sql_credentials["port"],
                                         sql_credentials["database"]))

In [ ]:
metadata = MetaData()
fish_crops = Table('lati_fish_detections', metadata, autoload=True, autoload_with=sql_engine)
lice_crops = Table('lati_fish_detections_lice_annotations', metadata, autoload=True,
                   autoload_with=sql_engine)

# inner join on fish crop id
# TODO @Thomas debug this
query = select([fish_crops.c.image_key, lice_crops.c.lice_bbox_list]) \
    .select_from(lice_crops.join(fish_crops, lice_crops.c.lati_fish_detections_id == fish_crops.c.id)) \
    .where(and_(lice_crops.c.is_skipped == False,
           fish_crops.c.site_id == 23,
           fish_crops.c.pen_number == 2))


In [ ]:
results = []
with sql_engine.connect() as conn:
    for row in conn.execute(query):
        results.append((row[0], row[1]))

In [ ]:
print(len(results))

In [ ]:
from tqdm import tqdm

In [ ]:
for r in tqdm(results):
    key = r[0].replace('dev', '/root/data/lice-data/crops')
    destination = key.replace('.jpg', '_cogito.json')
    try: 
        with open(destination, 'w') as f:
            json.dump(r[1], f)
    except:
        continue